In [1]:
from dotenv import load_dotenv
import os
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.memory import ChatMemoryBuffer

In [2]:
load_dotenv()
# API_KEY = os.getenv('OPENAI_API_KEY1')

True

In [3]:
# check if storage already exists
PERSIST_DIR = "./storage"
if os.path.exists(PERSIST_DIR):
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
else:
    print("No storage found, creating new index")
index

In [61]:
query_engine = index.as_query_engine()
initial_query = """
            Please provide a list of the file names of the documents you have stored
            """
response = query_engine.query(initial_query)
print(response.response)

The file names of the documents stored are:
- The Count of Monte Cristo.pdf
- 2city12p.pdf


In [67]:
import re

output = response.response+'\n'

# Extract file names using regular expressions
file_names = re.findall(r'- (.+)\n', output)
print(file_names)

['The Count of Monte Cristo.pdf', '2city12p.pdf']


In [75]:
books = []
for file in file_names:
    doc_query = f"""
        Just provide the title of the document. 
        For document {file} in your stored documents, please provide the title.
        Do not give any text such as "The title is" or "The title of the document is".
    """
    doc_title = query_engine.query(doc_query)
    doc_query = f"""
        Just provide the author of the document. 
        For document {doc_title} in your stored documents, please provide the author.
        Do not give any text such as "The author is" or "The author of the document is".
    """
    doc_author = query_engine.query(doc_query)
    books.append({'title': doc_title.response, 'author': doc_author.response})
    # print(f'{doc_title}, {doc_author}')
print(books)

[{'title': 'The Count of Monte Cristo', 'author': 'Alexandre Dumas'}, {'title': 'A Tale of Two Cities', 'author': 'Charles Dickens'}]


In [80]:
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(f"""
                   You are a chatbot, able to have normal interactions, as well as talk
                   about the books in this list: {books}
                   You may not talk about anything else."""
    ),
)

In [77]:
response = chat_engine.chat("Hello!")
response

AgentChatResponse(response='Hello! How can I assist you today?', sources=[ToolOutput(content='[NodeWithScore(node=TextNode(id_=\'abb747a2-77c7-45d7-b541-4ced9e47df9c\', embedding=None, metadata={\'page_label\': \'35\', \'file_name\': \'2city12p.pdf\', \'file_path\': \'/home/gtrigo/apps/sources/ai/tutorial_ai_git/data/2city12p.pdf\', \'file_type\': \'application/pdf\', \'file_size\': 1316140, \'creation_date\': \'2024-12-29\', \'last_modified_date\': \'2024-12-29\'}, excluded_embed_metadata_keys=[\'file_name\', \'file_type\', \'file_size\', \'creation_date\', \'last_modified_date\', \'last_accessed_date\'], excluded_llm_metadata_keys=[\'file_name\', \'file_type\', \'file_size\', \'creation_date\', \'last_modified_date\', \'last_accessed_date\'], relationships={<NodeRelationship.SOURCE: \'1\'>: RelatedNodeInfo(node_id=\'2a55e582-7ddf-4c46-b0c0-ef65c8de8a3d\', node_type=\'4\', metadata={\'page_label\': \'35\', \'file_name\': \'2city12p.pdf\', \'file_path\': \'/home/gtrigo/apps/sources/ai/

In [81]:
response = chat_engine.chat("What are you capable of?")
print(response)

I am a chatbot designed to discuss and provide information about the books "The Count of Monte Cristo" by Alexandre Dumas and "A Tale of Two Cities" by Charles Dickens. Feel free to ask me questions or engage in conversations related to these books!


In [82]:
response = chat_engine.chat("Give me a summary of Tale of Two Cities")
print(response)

"A Tale of Two Cities" by Charles Dickens is a historical novel set in London and Paris before and during the French Revolution. The story follows the lives of several characters, including Charles Darnay, a French aristocrat who renounces his title, and Sydney Carton, a dissolute English lawyer. The novel explores themes of resurrection, sacrifice, and the impact of social injustice. As the revolution unfolds in France, the characters' lives become intertwined in a complex and dramatic narrative that culminates in a powerful and emotional conclusion. The novel is known for its vivid depiction of the turbulent historical period and its exploration of human nature and redemption.


In [83]:
response = chat_engine.chat("what is the time period for this book?")
print(response)

"A Tale of Two Cities" by Charles Dickens is set during the late 18th century, specifically during the years leading up to and during the French Revolution. The novel begins in 1775 and covers the period of significant social and political upheaval in both London and Paris.


In [84]:
response = chat_engine.chat("are people travelling between those cities or is this about parallel lives?")
print(response)

In "A Tale of Two Cities," the characters are connected through their interactions and relationships that span both London and Paris. The novel explores the lives of characters who have connections to both cities, highlighting the parallels and contrasts between their experiences in each location. While there is travel between the two cities in the story, the focus is more on how the characters' lives intersect and intertwine despite the physical distance between London and Paris.


In [85]:
response = chat_engine.chat("Give me a summary of the Count of Monte Cristo")
print(response)

"The Count of Monte Cristo" by Alexandre Dumas is a classic novel that tells the story of Edmond Dantès, a young sailor who is falsely accused of treason and imprisoned in the Château d'If. During his time in prison, Dantès befriends a fellow inmate who educates him and reveals the location of a hidden treasure on the island of Monte Cristo. After a daring escape, Dantès acquires the treasure, adopts the persona of the Count of Monte Cristo, and sets out on a mission of revenge against those who wronged him.

As the Count of Monte Cristo, Dantès uses his newfound wealth and intelligence to manipulate the lives of his enemies, exposing their secrets and orchestrating their downfall. Along the way, he encounters love, betrayal, and moral dilemmas that challenge his sense of justice and vengeance. The novel explores themes of justice, redemption, and the consequences of seeking revenge.


In [86]:
response = chat_engine.chat("is this book about revenge?")
print(response)

Yes, "The Count of Monte Cristo" by Alexandre Dumas is a novel that revolves around the theme of revenge. The main character, Edmond Dantès, is wrongfully imprisoned and seeks vengeance against those who betrayed him. The story follows Dantès as he meticulously plans and executes his revenge on those who caused his suffering, all while assuming the identity of the wealthy and enigmatic Count of Monte Cristo. The novel delves into the complexities of revenge, exploring the moral implications and consequences of seeking retribution.


In [87]:
response = chat_engine.chat("is it worthy for the main character to obtain revenge or is it too costly?")
print(response)

In "The Count of Monte Cristo," the theme of revenge is central to the story, and the novel raises questions about the morality and consequences of seeking vengeance. While Edmond Dantès' pursuit of revenge allows him to right the wrongs done to him and bring justice to those who betrayed him, it also comes at a great cost. Throughout his quest for vengeance, Dantès faces moral dilemmas, internal conflicts, and personal sacrifices that challenge his sense of justice and humanity.

Ultimately, the novel portrays revenge as a double-edged sword, showcasing both the satisfaction of obtaining justice and the destructive nature of harboring hatred and seeking retribution. The story highlights the complexities of revenge and its impact on both the avenger and those targeted, leaving readers to ponder whether the price of revenge is worth the toll it takes on the individual seeking it.


In [88]:
response = chat_engine.chat("is there an alternative that would provide more satisfaction at a lower cost?")
print(response)

In "The Count of Monte Cristo," while revenge plays a significant role in the story, there are alternative paths that characters could have taken to seek satisfaction or justice without resorting to vengeance. For example, forgiveness, redemption, and reconciliation are themes that could have been explored as alternatives to revenge.

Forgiveness: Instead of seeking revenge, characters could have chosen to forgive those who wronged them, letting go of anger and resentment in favor of moving forward with their lives.

Redemption: Characters could have focused on redeeming themselves or helping others find redemption, seeking to make amends for past mistakes and finding inner peace through positive actions.

Reconciliation: Rather than seeking to destroy their enemies, characters could have pursued reconciliation and understanding, working towards resolving conflicts and building bridges instead of burning them.

These alternatives, while not as dramatic or immediate as seeking revenge, 